In [120]:
from copy import deepcopy
from operator import itemgetter

class Literal:
    
    def __init__(self, name, sign = True):
        self.name = name
        self.sign = sign
    
    #symbols to be compared just by name:
    def __eq__(self, other):
        return other.name == self.name
    
    def __neg__(self):
        neg = Literal(self.name, sign = not self.sign)
        return neg
    
    #make the class a hashable type
    def __hash__(self):
        return hash(self.name)
    
    def __str__(self):       
        if self.sign: return self.name            
        return "-" + self.name

In [121]:
sat_model = {}
Sat = Literal("satisfied")

def DPLL(clauses, symbols, model):
    
    for clause in clauses:
        # store symbols we can remove from clause
        remove = set()
        for literal in clause:
            if literal.name in model:
                # Literal in model with same sign as in clause?                
                if literal.sign == model[literal.name] : 
                    clause.add(Sat)                    
                    break
                # are there other literals to be tried
                elif len(clause) > 1:
                    remove.add(literal)
                # no more literals in clause, clause not satisfied
                else:
                    return False
        # remove every "False"
        clause -= remove
                
    #have all clauses been satisfied?    
    if all([Sat in clause for clause in clauses]):
        global sat_model      
        sat_model = model
        return True

    
    if symbols: 
        new_sym = symbols.pop(0)
        model[new_sym.name] = True
        truth_val1 = DPLL(deepcopy(clauses), deepcopy(symbols), model)
        model[new_sym.name] = False 
        truth_val2 = DPLL(deepcopy(clauses), deepcopy(symbols), model)
        
        return truth_val1 or truth_val2
    
    #by this point exhaustive search failed, model is not satisfiable
    return False
    
    

def DPLLSatisfiable(KB):
    #extract all symbols. We want to have each symbol once 
    symbols = {}
    for clause in KB:
        for sym in clause:
            if sym in symbols: symbols[sym] += 1                
            else: symbols[sym] = 1
    #sorting symbols by frequency
    symbols = sorted([ (key, count) for key, count in symbols.items()], reverse = True, key= itemgetter(1) )
    symbols = [pair[0] for pair in symbols]
    
    
    sat = DPLL(KB, list(symbols), {})
    if sat:
        # all literals in KB
        for symbol in set().union(*KB):
            if symbol.name not in sat_model:
                sat_model[symbol.name] = "Free"
            else:
                sat_model[symbol.name] = str(sat_model[symbol.name])

        return sat, sat_model  
    return sat    
        

S1: A ⇔ (B ∨ E).

S2: E ⇒ D.

S3: C ∧ F ⇒ ¬B.

S4: E ⇒ B.

S5: B ⇒ F.

S6: B ⇒ C

Writing this in CNF:

(A ⇔ (B ∨ E)) ∧ (E ⇒ D) ∧ (C ∧ F ⇒ ¬B) ∧ (E ⇒ B) ∧ (B ⇒ F) ∧ (B ⇒ C)

(A ⇒ B ∨ E ) ∧ (B ∨ E ⇒ A) ∧ (-E ∨ D) ∧ (-C ∨ -F ∨ -B) ∧ (-E ∨ B) ∧ (-B ∨ F) ∧ (-B ∨ C)

In [122]:
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

KB = [{-A, B, E}, {-B, A}, {-E, A}, {-E, D}, 
      {-C, -F, -B}, {-E, B}, {-B, F}, {-B, C}]

In [123]:
DPLLSatisfiable(KB)

(True,
 {'A': 'False',
  'B': 'False',
  'C': 'False',
  'D': 'Free',
  'E': 'False',
  'F': 'False'})